In [1]:
# 抽取 实体-关系词二元组
import json
data=json.loads(open("result/es_top20_train.json",encoding="utf8").read())

In [20]:
data[2]

{'id': 'TR002941',
 'question': '圆明园的开放时间是什么时候？',
 'attribute': ['没有找到该问题对应的知识'],
 'top20': [['5029130', '时间是什么 ||| 书名 ||| 时间是什么'],
  ['5029129', '时间是什么 ||| 别名 ||| 时间是什么'],
  ['5181735', '什么时候 ||| 别名 ||| 什么时候'],
  ['12223520', '什么时候明朝初年 ||| 别名 ||| 什么时候明朝初年'],
  ['5029136', '时间是什么 ||| 开本 ||| 32'],
  ['12223521', '什么时候明朝初年 ||| 中文名 ||| 什么时候明朝初年'],
  ['465265', '圆明园遗址公园 ||| 开放时间 ||| 1988年'],
  ['16595056', '什么时候说什么话 ||| 别名 ||| 什么时候说什么话'],
  ['16595061', '什么时候说什么话 ||| 出版时间 ||| 2008.6'],
  ['5181736', '什么时候 ||| 中文名称 ||| 什么时候'],
  ['8765481', '爱情从什么时候开始 ||| 别名 ||| 爱情从什么时候开始'],
  ['17544776', '忘了什么时候说过不放手 ||| 别名 ||| 忘了什么时候说过不放手'],
  ['28793305', '什么时候我们才学会放手 ||| 别名 ||| 什么时候我们才学会放手'],
  ['8765482', '爱情从什么时候开始 ||| 中文名 ||| 爱情从什么时候开始'],
  ['17544777', '忘了什么时候说过不放手 ||| 中文名 ||| 忘了什么时候说过不放手'],
  ['28793306', '什么时候我们才学会放手 ||| 中文名 ||| 什么时候我们才学会放手'],
  ['3372028', '都什么时候了 ||| 别名 ||| 都什么时候了'],
  ['36489788', '什么时候吻她 ||| 别名 ||| 什么时候吻她'],
  ['28113773', '开放人文·真科学：它是什么，它指什么 ||| 出版时间 ||| 2008-12-1'],
  ['176

In [ ]:
def generate_prompt_QuerySummary(related_text, query: str, preprompt="") -> str:
    prompt_template = """已知问题:
        {question}，请提取问题的主宾二元组，格式：xx|||xx"""

    prompt = preprompt + prompt_template.replace("{question}", query).replace("{context}", related_text)
    return prompt

In [107]:
data_tuple=[]
for unit in data:
    if '没有找到该问题对应的知识' not in unit["attribute"]:
        # print(unit)
        question = unit["question"]
        answer = ";".join(["|||".join([t.strip() for t in k.split("|||")[:2]]) for k in unit["attribute"]])
        data_tuple.append({"question":question,"instruction":"","answer":answer,"task_type":"tuple_extract"})

842

In [25]:
data_tuple[2]

('周穆王在位多少年？', '周穆王|||在位时间')

In [96]:
# 抽取表格信息
def generate_prompt_QA(related_text, query: str, preprompt="") -> str:
    prompt_template = """基于以下表格信息，来回答用户的问题。
        如果无法从中得到答案，请说 "没有找到该问题对应的知识"，不允许在答案中添加编造成分，答案请使用中文。
        已知内容:
        {context}
        问题:
        '{question}'"""

    prompt = preprompt + prompt_template.replace("{question}", query).replace("{context}", related_text)
    return prompt


In [72]:
# step
i=16
generate_prompt_QA(str(";".join([k[1] for k in data[i]["top20"]])),data[i]["question"])

'基于以下已知信息，简洁和专业的来回答用户的问题。\n        如果无法从中得到答案，请说 "没有找到该问题对应的知识"，不允许在答案中添加编造成分，答案请使用中文。\n        已知内容:\n        富人的世界是什么样 ||| 别名 ||| 富人的世界是什么样;我是一个什么样的人 ||| 别名 ||| 我是一个什么样的人;什么样的人做什么样的事 ||| 别名 ||| 什么样的人做什么样的事;什么样的人做什么样的事 ||| 书名 ||| 什么样的人做什么样的事;《什么样的人赚什么样的钱》 ||| 别名 ||| 《什么样的人赚什么样的钱》;《什么样的人赚什么样的钱》 ||| 中文名 ||| 《什么样的人赚什么样的钱》;富人的世界是什么样 ||| 中文名 ||| 富人的世界是什么样;我是一个什么样的人 ||| 中文名 ||| 我是一个什么样的人;什么样的人有什么样的经历 ||| 别名 ||| 什么样的人有什么样的经历;是什么样的感情 ||| 别名 ||| 是什么样的感情;什么样的人有什么样的经历 ||| 中文名 ||| 什么样的人有什么样的经历;娶什么样的女人，嫁什么样的男人 ||| 别名 ||| 娶什么样的女人，嫁什么样的男人;是什么样的感情 ||| 中文名 ||| 是什么样的感情;井上雄 ||| 人物类型 ||| 动漫人物;什么样的人能干，什么样的人不能干 ||| 别名 ||| 什么样的人能干，什么样的人不能干;你是什么样的父母 ||| 别名 ||| 你是什么样的父母;什么样的人生是值得追求的 ||| 别名 ||| 什么样的人生是值得追求的;你是什么类型的父母 ||| 别名 ||| 你是什么类型的父母;你是什么类型的父母 ||| 书名 ||| 你是什么类型的父母;金玉彬 ||| 是什么样的人 ||| 火\n        问题:\n        \'井上雄是什么样的人物类型？\''

In [73]:
top20 = [k[1] for k in data[i]["top20"]]
set(top20)&set(data[i]["attribute"])

{'井上雄 ||| 人物类型 ||| 动漫人物'}

In [108]:
data_table_extraction=[]
for unit in data:
    top20 = [k[1] for k in unit["top20"]]
    if not set(top20)&set(unit["attribute"]):
        answer = "没有找到该问题对应的知识"
    else:
        answer = ";".join(set(top20)&set(unit["attribute"]))
    data_table_extraction.append({"question":unit["question"],"instruction":top20,"answer":answer,"task_type":"table_extract"})
        

In [84]:
sums=0
for d in data_table_extraction:
    if d[2]!="没有找到该问题对应的知识":
        sums+=1

In [85]:
sums

335

In [109]:
# 正样本构造
import random

data_table_extraction2=[]
for unit in data:
    top20 = [k[1] for k in unit["top20"]]
    if not set(top20)&set(unit["attribute"]) and '没有找到该问题对应的知识' not in unit["attribute"]:
        answer = ";".join(unit["attribute"])
        top20=(unit["attribute"]+top20)[:20]
        random.shuffle(top20)
        data_table_extraction2.append({"question":unit["question"],"instruction":top20,"answer":answer,"task_type":"table_extract"})

In [111]:
data_sft=data_tuple+data_table_extraction+data_table_extraction2

In [114]:
random.shuffle(data_sft)

In [115]:
data_sft[0]

{'question': '孙莉在哪一天完婚的？',
 'instruction': ['走失在哪一个夏天 ||| 别名 ||| 走失在哪一个夏天',
  '我的春天在哪里 ||| 别名 ||| 我的春天在哪里',
  '天堂在哪? ||| 中文名 ||| 天堂在哪?',
  '教育在哪里：一个校长的365天 ||| 别名 ||| 教育在哪里：一个校长的365天',
  '我想结婚的时候你在哪 ||| 别名 ||| 我想结婚的时候你在哪',
  '天的阳光在哪？ ||| 中文名 ||| 天的阳光在哪?',
  '我的摩天轮，你在哪？ ||| 别名 ||| 我的摩天轮，你在哪？',
  '我想结婚的时候你在哪 ||| 中文名 ||| 我想结婚的时候你在哪',
  '天堂在哪? ||| 小说进度 ||| 已完结',
  '我的春天在哪里 ||| 中文名 ||| 我的春天在哪里',
  '完美的一天(孙燕姿演唱专辑) ||| 别名 ||| 完美的一天',
  '教育在哪里：一个校长的365天 ||| 书名 ||| 教育在哪里：一个校长的365天',
  '下一个天亮你在哪里 ||| 别名 ||| 下一个天亮你在哪里',
  '天的阳光在哪？ ||| 别名 ||| 天的阳光在哪？',
  '天堂在哪？ ||| 别名 ||| 天堂在哪？',
  '孙莉 ||| 结婚纪念日 ||| 2004年3月8日',
  '下一个天亮你在哪里 ||| 中文名 ||| 下一个天亮你在哪里',
  '我想结婚的时候你在哪 ||| 中文名称 ||| 我想结婚的时候你在哪',
  '天堂在哪? ||| 别名 ||| 天堂在哪?',
  '走失在哪一个夏天 ||| 中文名 ||| 走失在哪一个夏天'],
 'answer': '孙莉 ||| 结婚纪念日 ||| 2004年3月8日',
 'task_type': 'table_extract'}

In [117]:
with open("train_sft.json","w",encoding="utf8" ) as f:
    for unit in data_sft[:int(0.8*(len(data_sft)))]:
        f.write(json.dumps(unit,ensure_ascii=False))
        f.write("\n")

with open("dev_sft.json","w",encoding="utf8" ) as f:
    for unit in data_sft[int(0.8*(len(data_sft))):]:
        f.write(json.dumps(unit,ensure_ascii=False))
        f.write("\n")